# Dataset Translation and Preparations to Thai Language By #AI


### Utilize Requirements package

In [2]:
import asyncio
from ollama import AsyncClient
from datasets import load_dataset
import pandas as pd

### Functions translate text to thai

In [3]:

async def translate_to_thai(text):
    """
    Translates text from any language into Thai using the llama3.1 model.

    Args:
        text (str): The text to be translated.

    Returns:
        str: The translated Thai text.
    """
    res = await AsyncClient().generate(model="llama3.1:latest",
                   system="""# System
        You are a professional translator specializing in translating text from any language into Thai. Your goal is to produce natural-sounding Thai translations that use common, everyday language and phrases.
        
        # Instructions:
        * Translate the text accurately and completely.
        * Maintain Nouns In English
        * Keep Sticky and Correctly Thai Grammar 
        * Employ real-world usage and common phrases. Think like a native Thai speaker and choose words and sentences that they would use in everyday conversations.
        Maintain the intended tone and meaning of the original text. If the original text is humorous, informal, or formal, reflect that in your translation.

        # Output Format
        translations output

        # Example 
        Text to translate : "Generate a list of ten items a person might need for a camping trip"
        สร้างรายการที่จำเป็น 10 รายการสำหรับการท่องเที่ยวในพื้นที่ป่าไม้
        
        Text to translate : "Explain the use of word embeddings in Natural Language Processing"
        อธิบายการใช้คำที่ฝังอยู่ในการประมวลผลภาษาธรรมชาติ
        # Rule
        * Answer Only Answer Without Additional explanation
        """, 
        prompt=f"Text to translate : \"{text}\""
    )

    return res['response']



### Loading Dataset
dataset can change to difference repo from huggingface

In [4]:
dataset = load_dataset("yahma/alpaca-cleaned",split="train")

### Convert Dataset to Dict

In [5]:
df = dataset.to_dict()
dataframe = pd.DataFrame(df)
dataframe.head()

,output,input,instruction
0,1. Eat a balanced and nutritious diet: Make su...,,Give three tips for staying healthy.
1,"The three primary colors are red, blue, and ye...",,What are the three primary colors?
2,An atom is the basic building block of all mat...,,Describe the structure of an atom.
3,There are several ways to reduce air pollution...,,How can we reduce air pollution?
4,I had to make a difficult decision when I was ...,,Pretend you are a project manager of a constru...


### Check Columns

In [7]:
dataframe.columns

Index(['output', 'input', 'instruction'], dtype='object')

### Slice for Trial

In [8]:
dataframe1 = dataframe[0:10]
dataframe1

,output,input,instruction
0,1. Eat a balanced and nutritious diet: Make su...,,Give three tips for staying healthy.
1,"The three primary colors are red, blue, and ye...",,What are the three primary colors?
2,An atom is the basic building block of all mat...,,Describe the structure of an atom.
3,There are several ways to reduce air pollution...,,How can we reduce air pollution?
4,I had to make a difficult decision when I was ...,,Pretend you are a project manager of a constru...
5,The Commodore 64 was a highly successful 8-bit...,,"Write a concise summary of the following:\n""Co..."
6,The fraction 4/16 is equivalent to 1/4 because...,4/16,Explain why the following fraction is equivale...
7,"Sophie sat at her desk, staring blankly at the...",,Write a short story in third person narration ...
8,There are two spelling errors in the sentence....,He finnished his meal and left the resturant,Evaluate this sentence for spelling and gramma...
9,"Julius Caesar, the Roman Military general, and...",,How did Julius Caesar die?


### Create Task for Asyncio

In [124]:
async def translate_column_ollama(dataframe1, column_name):
    tasks = [translate_to_thai(row) for row in dataframe1[column_name] if len(row) > 0]
    translations = await asyncio.gather(*tasks)
    dataframe1.loc[dataframe1[column_name].apply(len) > 0, column_name] = translations

### Excute Task and Translations

In [125]:
await asyncio.gather(
        translate_column_ollama(dataframe1, 'input'),
        translate_column_ollama(dataframe1, 'output'),
        translate_column_ollama(dataframe1, 'instruction')
    )
print('finally')

C:\Users\origi\AppData\Local\Temp\ipykernel_4644\3257845081.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe1.loc[dataframe1[column_name].apply(len) > 0, column_name] = translations


finally


### Check Output

In [131]:
dataframe1.head()

,output,input,instruction
0,สร้างรายการที่จำเป็น 3 อย่างสำหรับชีวิตที่ดีที...,,ให้สามคำแนะนำสำหรับการดูแลสุขภาพ
1,สีหลัก 3 สีที่สำคัญคือ แดง เขียว น้ำเงิน สีต่า...,,อะไรคือสีสามสีที่เป็นฐาน
2,แอตรอน (Atom) คือปัจจัยพื้นฐานของวัสดุและประกอ...,,อธิบายโครงสร้างของอะตอม
3,ข้อควรทำ 6 ข้อในการลดมลพิษทางอากาศ ได้แก่:\n\n...,,วิธีการลดมลพิษทางอากาศได้อย่างไร?
4,ฉันต้องตัดสินใจที่ยากลำบากเมื่อทำงานเป็นผู้จัด...,,เมื่อครั้งหนึ่งมีงานก่อสร้างที่ต้องดำเนินการใน...


### Convert Back to Dataset

In [127]:
from datasets import Dataset
dataset = Dataset.from_pandas(df=dataframe1)

### Convert Dataset into Training Template

In [128]:
chat = """<|im_start|>system
{SYSTEM}<|im_end|>
<|im_start|>user
{INPUT}<|im_end|>
<|im_start|>assistant
{OUTPUT}<|im_end|>"""

# EOS_TOKEN = tokenizer.eos_token
def Formating(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = chat.format(SYSTEM=instruction, INPUT=input, OUTPUT=output) #+ EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

### Map Template with Dataset

In [129]:
dataset = dataset.map(Formating,batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

### Result

In [133]:
print(dataset['text'][2])

<|im_start|>system
อธิบายโครงสร้างของอะตอม<|im_end|>
<|im_start|>user
<|im_end|>
<|im_start|>assistant
แอตรอน (Atom) คือปัจจัยพื้นฐานของวัสดุและประกอบด้วยสสารสามชนิด ได้แก่ โปรตอน นิวตรอน และอิเล็กตรอน โครงสร้างของแอตรอนสามารถอธิบายได้ว่าเป็นศูนย์กลางที่มีโปรตอนและนิวตรอนอยู่ในบริเวณนั้น และมีอิเล็กตรอนห้าบรรจุเก็บพอดี (electron cloud) ตลอดออกไปจากศูนย์กลางของแอตรอน โปรตอนเป็นอิเล็กทรอนชนิดที่มีประจุกำลังบวก ส่วนนิวตรอนที่ไม่มีประจุขั้น

บริเวณศูนย์กลางของแอตรอนที่ประกอบด้วยโปรตอนและนิวตรอน โดยโปรตอนดังกล่าวเป็นกระแสไฟฟ้าบวกที่มีผลกระทบต่อการทำงานของแอตรอน ส่วนนิวตรอนไม่มีสภาวะประจุใดๆ ทั้งสิ้น และอยู่ห่างออกไปจากศูนย์กลางของแอตรอน

บริเวณรอบ ๆ ศูนย์กลางของแอตรอนที่ประกอบด้วยกลุ่มอิเล็กตรอน (electron cloud) ส่วนใหญ่ๆ อิเล็กตรอนเหล่านี้จะมีประจุไฟฟ้าลบ มีการเคลื่อนที่อย่างต่อเนื่อง และแบ่งออกเป็นชั้นหรือวงโคจร (shells หรือ orbitals) โดยแต่ละชั้นสามารถเก็บอิเล็กตรอนได้จำนวนหนึ่ง

ส่วนชั้นภายนอกที่เรียกว่า ชั้นวาเลนซ์ (valence shell) จะมีอิทธิพลต่อสมบัติทางเคมีของแอตรอนดังกล่าว โดยเฉพาะอย